# 1.机器阅读理解
## 评估指标
1 精准匹配度(Exact Match, EM)：计算预测结果与标准答案是否完全匹配
2 模糊匹配度(F1)：计算预测结果与标准答案之间字符的匹配度  
e.g:  
模型预测结果：北京  
真实标签结果：北京天安门

$$EM = 0; $$
$$\text{Precision} = \frac{2}{2}, \quad \text{Recall} = \frac{2}{5}, \quad F1 = \frac{2 \times \frac{2}{2} \times \frac{2}{5}}{\frac{2}{5} + \frac{2}{2}} = \frac{4}{7};$$

# 2.基于Transformers的解决方案
## 数据预处理
- 数据处理格式：
  [CLS] Question [SEP] Context [SEP]
- 定位答案位置：
  start_positions 和 end_positions (这里的position是Token的位置，不是字符位置)
  offset_mapping (将Token映射到字符位置)
- Context过长解决策略：
  1. 直接截断，实现比较简单，但是会损失答案靠后的信息。
  2. 滑动窗口，实现较复杂，但是会保留答案靠后的信息。
## 模型结构
- *ModelForQuestionAnswering
```python
class BertForQuestionAnswering(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels #这里的num_labels是2 起始位置和终点位置
        self.bert = BertModel(config, add_pooling_layer=False)
        self.qa_outputs = nn.Linear(config.hidden_size, config.num_labels) 
        # Initialize weights and apply final processing
        self.post_init()

    @auto_docstring
    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        start_positions: Optional[torch.Tensor] = None,
        end_positions: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[tuple[torch.Tensor], QuestionAnsweringModelOutput]:
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)  #[batch_size, seq_len, 2]
        start_logits, end_logits = logits.split(1, dim=-1) #[batch_size, seq_len, 1]
        start_logits = start_logits.squeeze(-1).contiguous() #[batch_size, seq_len]
        end_logits = end_logits.squeeze(-1).contiguous() 

        total_loss = None
        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions = start_positions.clamp(0, ignored_index) #删去[SEP]标签
            end_positions = end_positions.clamp(0, ignored_index)

            loss_fct = CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2

        if not return_dict:
            output = (start_logits, end_logits) + outputs[2:]
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
```

# 3.基于截断策略的实现

## steps1 导入相关包

In [25]:
from datasets import  load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator

## steps2 数据集加载

In [26]:
dataset = load_dataset('../dataset/cmrc2018')
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [27]:
dataset['train'][0]

{'id': 'TRAIN_186_QUERY_0',
 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
 'question': '范廷颂是什么时候被任为主教的？',
 'answers': {'text': ['1963年'], 'answer_start': [30]}}

## steps3 数据预处理

In [28]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
tokenizer

BertTokenizerFast(name_or_path='hfl/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [29]:
sample_dataset = dataset['train'].select(range(10))
# datasets.Dataset的列返回的是datasets.Column对象 这里需要转换成list格式
questions = list(sample_dataset["question"])
contexts = list(sample_dataset["context"])
# answers = list(sample_dataset['answers'])
sample_dataset, questions, contexts

(Dataset({
     features: ['id', 'context', 'question', 'answers'],
     num_rows: 10
 }),
 ['范廷颂是什么时候被任为主教的？',
  '1990年，范廷颂担任什么职务？',
  '范廷颂是于何时何地出生的？',
  '1994年3月，范廷颂担任什么职务？',
  '范廷颂是何时去世的？',
  '安雅·罗素法参加了什么比赛获得了亚军？',
  'Russell Tanoue对安雅·罗素法的评价是什么？',
  '安雅·罗素法合作过的香港杂志有哪些？',
  '毕业后的安雅·罗素法职业是什么？',
  '岬太郎在第一次南葛市生活时的搭档是谁？'],
 ['范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任

In [30]:
tokenized_sample = tokenizer(text=questions,
                            text_pair=contexts,
                            return_offsets_mapping=True,
                            max_length=384,
                            truncation="only_second",
                            padding="max_length")
tokenized_sample.data.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping'])

In [31]:
tokenized_sample['offset_mapping'][0]  #token对具体词的映射

[(0, 0),
 (0, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10),
 (10, 11),
 (11, 12),
 (12, 13),
 (13, 14),
 (14, 15),
 (0, 0),
 (0, 1),
 (1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10),
 (10, 11),
 (11, 12),
 (12, 13),
 (13, 14),
 (14, 15),
 (15, 16),
 (16, 17),
 (17, 18),
 (18, 19),
 (19, 20),
 (20, 21),
 (21, 22),
 (22, 23),
 (23, 24),
 (24, 25),
 (25, 26),
 (26, 27),
 (27, 28),
 (28, 29),
 (29, 30),
 (30, 34),
 (34, 35),
 (35, 36),
 (36, 37),
 (37, 38),
 (38, 39),
 (39, 40),
 (40, 41),
 (41, 45),
 (45, 46),
 (46, 47),
 (47, 48),
 (48, 49),
 (49, 50),
 (50, 51),
 (51, 52),
 (52, 53),
 (53, 54),
 (54, 55),
 (55, 56),
 (56, 57),
 (57, 58),
 (58, 59),
 (59, 60),
 (60, 61),
 (61, 62),
 (62, 63),
 (63, 67),
 (67, 68),
 (68, 69),
 (69, 70),
 (70, 71),
 (71, 72),
 (72, 73),
 (73, 74),
 (74, 75),
 (75, 76),
 (76, 77),
 (77, 78),
 (78, 79),
 (79, 80),
 (80, 81),
 (81, 82),
 (82, 83),
 (83, 84),
 (84, 85),
 (85, 86),
 (86, 87

In [32]:
offsets_mapping = tokenized_sample.pop("offset_mapping")  

In [33]:
print(tokenized_sample.sequence_ids(0))  #每个token对应哪一个句子， None是[CLS]或[SEP]

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [34]:
for idx, offset in enumerate(offsets_mapping):
    answers = sample_dataset['answers'][idx]
    start_char = answers['answer_start'][0]
    end_char = start_char + len(answers['text'][0])
    
    context_start = tokenized_sample.sequence_ids(idx).index(1) #index获得list中第一个为1的索引 即context的开始索引
    context_end   = tokenized_sample.sequence_ids(idx).index(None, context_start)-1 #从context_start开始，找到第一个为None的索引，减1得到context的结束索引
    
    # 判断答案是否超出了max_length；
    if offset[context_end][1] < start_char or offset[context_start][0] > end_char: #判断答案是否超出了max_length；
        start_token_pos = 0
        end_token_pos = 0
    else:
        start_token_pos = context_start  
        while start_token_pos < context_end and offset[start_token_pos][0] < start_char:
            start_token_pos += 1
        end_token_pos = context_end
        while end_token_pos >  context_start and offset[end_token_pos][1] > end_char:
            end_token_pos -= 1
        
    print(answers,start_char, end_char, context_start, context_end, start_token_pos, end_token_pos)
    print('token answer:', tokenizer.decode(tokenized_sample['input_ids'][idx][start_token_pos:end_token_pos+1]))

{'text': ['1963年'], 'answer_start': [30]} 30 35 17 382 47 48
token answer: 1963 年
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 382 53 70
token answer: 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理
{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 382 100 124
token answer: 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生
{'text': ['1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理'], 'answer_start': [548]} 548 598 17 382 0 0
token answer: [CLS]
{'text': ['范廷颂于2009年2月22日清晨在河内离世'], 'answer_start': [759]} 759 780 12 382 0 0
token answer: [CLS]
{'text': ['《全美超级模特儿新秀大赛》第十季'], 'answer_start': [26]} 26 42 21 382 47 62
token answer: 《 全 美 超 级 模 特 儿 新 秀 大 赛 》 第 十 季
{'text': ['有前途的新面孔'], 'answer_start': [247]} 247 254 20 382 232 238
token answer: 有 前 途 的 新 面 孔
{'text': ['《Jet》、《东方日报》、《Elle》等'], 'answer_start': [706]} 706 726 20 382 0 0
token answer: [CLS]
{'text': ['售货员'], 'answer_start': [202]} 202 205 18 382 205 207
token answer: 售 货 员
{'text': ['大

In [35]:
def process_func(example):
    tokenized_dataset = tokenizer(
        text = list(example['question']),
        text_pair = list(example['context']),
        return_offsets_mapping = True,
        truncation = True,
        padding = 'max_length',
        max_length = 384,
    )
    offset_mapping = tokenized_dataset.pop("offset_mapping")
    start_positions = []
    end_positions = []
    for idx, offset in enumerate(offset_mapping):
        answers = example['answers'][idx]
        start_char = answers['answer_start'][0]
        end_char   = start_char + len(answers['text'][0])

        context_start = tokenized_dataset.sequence_ids(idx).index(1)
        context_end   = tokenized_dataset.sequence_ids(idx).index(None, context_start)

        if offset[context_end][1] < start_char or offset[context_start][0] > end_char:
            start_token_pos = 0
            end_token_pos = 0
        else:
            start_token_pos = context_start
            while start_token_pos < context_end and offset[start_token_pos][0] < start_char:
                start_token_pos += 1
            end_token_pos = context_end
            while end_token_pos >  context_start and offset[end_token_pos][1] > end_char:
                end_token_pos -= 1
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
    tokenized_dataset['start_positions'] = start_positions
    tokenized_dataset['end_positions'] = end_positions
    return tokenized_dataset

In [36]:
tokenized_dataset = dataset.map(process_func, batched=True, remove_columns=dataset['train'].column_names)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 1002
    })
})

## steps4 加载模型

In [37]:
model = AutoModelForQuestionAnswering.from_pretrained('hfl/chinese-macbert-base')
for param in model.bert.parameters():
    param.requires_grad = False

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## steps5 配置TrainingArguments

In [38]:
args = TrainingArguments(
    output_dir='models_for_Question',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy='epoch',
    run_name='runs',
    load_best_model_at_end=True,
    logging_steps=50,
)

## steps6 训练模型

In [39]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=DefaultDataCollator()
)

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.831700,1.469791
2,1.322200,1.171432
3,1.296500,1.156372


TrainOutput(global_step=1902, training_loss=1.99408299664719, metrics={'train_runtime': 451.7598, 'train_samples_per_second': 67.35, 'train_steps_per_second': 4.21, 'total_flos': 5962661340337152.0, 'train_loss': 1.99408299664719, 'epoch': 3.0})

## steps7 模型预测

In [41]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model,tokenizer=tokenizer, device=0)
pipe

Device set to use cuda:0


In [ ]:
# 我的GPU训练不动全参数 只能微调分类头 结果就是很差啊
pipe(question="如何评价《肖申克的救赎》？", context="《肖申克的救赎》是一个非常棒的电影")

{'score': 1.1614495633693878e-05, 'start': 12, 'end': 13, 'answer': '常'}